
# 🤖 Aula 10: Criando um Chatbot Empresarial Pronto para Produção

Neste notebook, vamos construir um chatbot seguro, escalável e pronto para uso em empresas.

## 📌 O que faremos:
1. Implementar segurança e proteção de dados do usuário.
2. Criar um banco de dados para armazenar conversas.
3. Integrar o chatbot com APIs corporativas.
4. Preparar o chatbot para deploy em produção.

---


In [ ]:

# Instale as bibliotecas necessárias
!pip install openai requests sqlite3 jwt logging


In [ ]:

import openai
import jwt
import logging
import sqlite3
import requests

# Defina sua chave de API (substitua 'SUA_CHAVE_AQUI' pela chave real)
openai.api_key = "SUA_CHAVE_AQUI"

# Configuração do log
logging.basicConfig(level=logging.INFO)


In [ ]:

# Função para gerar tokens JWT para autenticação
def gerar_token(usuario):
    token = jwt.encode({"user": usuario}, "chave_secreta", algorithm="HS256")
    return token

# Testando geração de token
print(gerar_token("admin"))


In [ ]:

# Instale as bibliotecas necessárias
!pip install openai requests sqlite3 pyjwt logging cryptography

import openai
import jwt
import logging
import sqlite3
import requests
from cryptography.fernet import Fernet

# Defina sua chave de API aqui (substitua 'SUA_CHAVE_AQUI' pela chave real)
openai.api_key = "SUA_CHAVE_AQUI"

# Configuração do log para auditoria de acessos
logging.basicConfig(filename="chatbot_log.log", level=logging.INFO, format="%(asctime)s - %(message)s")

# Geração e gerenciamento de criptografia para proteção de dados
chave_criptografia = Fernet.generate_key()
cipher_suite = Fernet(chave_criptografia)

# Função para criptografar mensagens antes de salvar
def criptografar_dados(dado):
    return cipher_suite.encrypt(dado.encode()).decode()

# Função para descriptografar mensagens quando necessário
def descriptografar_dados(dado_criptografado):
    return cipher_suite.decrypt(dado_criptografado.encode()).decode()

# Função para gerar tokens JWT para autenticação segura
def gerar_token(usuario):
    token = jwt.encode({"user": usuario}, "chave_secreta", algorithm="HS256")
    return token

# Função para validar tokens JWT
def validar_token(token):
    try:
        return jwt.decode(token, "chave_secreta", algorithms=["HS256"])
    except jwt.ExpiredSignatureError:
        return "Token expirado! Faça login novamente."
    except jwt.InvalidTokenError:
        return "Token inválido!"

# Criando banco de dados SQLite para armazenar conversas com segurança
def criar_banco():
    conexao = sqlite3.connect("chatbot_secure.db")
    cursor = conexao.cursor()
    cursor.execute(
        '''CREATE TABLE IF NOT EXISTS historico (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            usuario TEXT,
            mensagem TEXT
        )'''
    )
    conexao.commit()
    conexao.close()

# Função para armazenar mensagens criptografadas no banco de dados
def salvar_mensagem(usuario, mensagem):
    conexao = sqlite3.connect("chatbot_secure.db")
    cursor = conexao.cursor()
    mensagem_criptografada = criptografar_dados(mensagem)
    cursor.execute("INSERT INTO historico (usuario, mensagem) VALUES (?, ?)", (usuario, mensagem_criptografada))
    conexao.commit()
    conexao.close()

# Função para conectar com uma API corporativa segura (exemplo: API de estoque)
def buscar_estoque(produto):
    headers = {"Authorization": f"Bearer {gerar_token('admin')}"}
    url = f"https://api.empresa.com/estoque?produto={produto}"
    resposta = requests.get(url, headers=headers).json()
    return resposta

# Criando o banco seguro
criar_banco()

# Testando funcionalidades empresariais
usuario_token = gerar_token("funcionario123")
print("Token JWT:", usuario_token)
print("Validação de Token:", validar_token(usuario_token))
salvar_mensagem("funcionario123", "Pedido de consulta ao estoque de notebooks")
print("Resposta API de Estoque:", buscar_estoque("notebook"))


In [ ]:

# Função para buscar dados de uma API externa (Exemplo: suporte ao cliente)
def buscar_ticket(cliente_id):
    resposta = requests.get(f"https://api.suporte.com/tickets/{cliente_id}")
    return resposta.json()

# Testando a integração
# print(buscar_ticket("12345"))  # Exemplo de ID de cliente


In [ ]:

# Chatbot empresarial que usa banco de dados e autenticação
def chatbot_empresarial(usuario, pergunta):
    token = gerar_token(usuario)  # Gerando token para autenticação
    logging.info(f"Usuário autenticado: {usuario}")  # Log de autenticação

    # Salvando pergunta no histórico
    salvar_mensagem(usuario, pergunta)

    # Gerando resposta via OpenAI
    resposta = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": pergunta}]
    )["choices"][0]["message"]["content"]

    return resposta

# Testando o chatbot empresarial
print(chatbot_empresarial("admin", "Qual é o status do meu pedido?"))


In [ ]:

# Exemplo de monitoramento do chatbot
def monitorar_chatbot():
    try:
        resposta = chatbot_empresarial("admin", "Teste de monitoramento.")
        logging.info("Chatbot funcionando corretamente.")
        return resposta
    except Exception as e:
        logging.error(f"Erro no chatbot: {e}")
        return "Erro ao processar a solicitação."

# Testando monitoramento
print(monitorar_chatbot())



## 🎯 Desafio para os alunos!
1. Criar um chatbot que utilize autenticação e proteja dados do usuário.
2. Implementar um banco de dados para armazenar conversas e preferências.
3. Criar um script para deploy automatizado do chatbot.

---
